In [1]:
import numpy as np 
import pandas as pd
from pandasgui import show

### Data Import

In [2]:
dict_dfs_ori = pd.read_excel('data_ori.xlsx', sheet_name=None)    # get all worksheets
print(dict_dfs_ori.keys())

dict_keys(['Factor', 'ExpCase', 'WebExpCase'])


In [3]:
# 違規案件稽查編號
set_expSN = set(dict_dfs_ori['ExpCase']['稽查事件編號']) | set(dict_dfs_ori['WebExpCase']['稽查序號'])

# 建立完整資料表，含 target, features, others
df_ori = dict_dfs_ori['Factor'].copy()
df_ori['target'] = df_ori['稽查事件編號'].isin(set_expSN).astype('int')    # indexing with isin

In [4]:
del dict_dfs_ori, set_expSN

### Data Preprocess

* 將資料縱切 3 份：目標(target)、特徵(features)、其它(others)

In [5]:
print(df_ori.columns)

Index(['稽查事件編號', '營業縣市', '是否兼具製造業', '是否兼具輸入業', '是否兼具販售業', '是否兼具物流業', '餐飲業次業別數',
       '上市上櫃', '網路銷售', '導入HACCP', '導入ISO22000', '來源流向管理', '投保責任險', '倉儲', '實驗室',
       '標章分數', '資本額', '食添法規認知', '逾期食品法規認知', '有直營連鎖店', '有加盟連鎖店', '同一地址有多個餐飲場所',
       '有餐飲場所且有工廠製造場所', '有餐飲場所且委託他廠代工', '美食平台', '有無專技人員', '場所型式', '場所規模',
       '有無技術士證照', '有無供應學校', 'Group', 'target'],
      dtype='object')


In [6]:
cols_notFeat = ['稽查事件編號', 'Group', 'target']

ser_target = df_ori[cols_notFeat[-1]]
df_features = df_ori.drop(cols_notFeat, axis=1)    # inplace = False (default), return a copy
df_others = df_ori[cols_notFeat[:-1]]

In [7]:
del df_ori, cols_notFeat

* 檢視特徵有無缺失值
    1. isna() （同 isnull() ）檢查資料表每一個值是否為缺失值，輸出 Dataframe
    2. any(axis=0) 檢查每一欄是否有 True （ reduce the index ），輸出 Series
    3. sum() 加總 Series

In [8]:
print(df_features.isna().any(axis=0).sum())

0


* 特徵資料型別轉換

In [ ]:
# show(df_features.dtypes)

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer

from pandas.api.types import CategoricalDtype    # 自行建立 ordered category
cat_ord = CategoricalDtype(categories=None, ordered=True)    # 不設類別 (default)

In [ ]:
cols_to_int = ['資本額', ]
cols_to_cat = ['營業縣市', '場所型式', ]
cols_to_ord = ['餐飲業次業別數', '標章分數', '場所規模', ]

to_int = FunctionTransformer(lambda x: x.astype('uint32'))    # 0 to (2**32)-1
to_cat = FunctionTransformer(lambda x: x.astype('category'))
to_ord = FunctionTransformer(lambda x: x.astype(cat_ord))

col_trans = make_column_transformer(
    (to_int, cols_to_int), (to_cat, cols_to_cat), (to_ord, cols_to_ord), 
    remainder='drop',    # 丟棄剩餘欄位 (default)
    verbose_feature_names_out=False    # 去掉 transformer 前綴
).set_output(transform='pandas')

df_features_nc = col_trans.fit_transform(df_features)


In [40]:
from pandas.api.types import is_integer_dtype

In [ ]:
df_features_b = df_features.drop(labels=df_features_nc.columns, axis=1)

df_features_b = df_features_b.apply(
    # conditional expression
    lambda col: col.astype('uint8') if is_integer_dtype(col) else col.map({'Y':1, 'N':0}).astype('uint8')
)


,是否兼具製造業,是否兼具輸入業,是否兼具販售業,是否兼具物流業,上市上櫃,網路銷售,導入HACCP,導入ISO22000,來源流向管理,投保責任險,...,逾期食品法規認知,有直營連鎖店,有加盟連鎖店,同一地址有多個餐飲場所,有餐飲場所且有工廠製造場所,有餐飲場所且委託他廠代工,美食平台,有無專技人員,有無技術士證照,有無供應學校
0,0,0,0,0,0,0,0,0,1,1,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0,1,1,...,1,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,1,0,1,1,...,1,0,0,1,1,0,1,1,1,0
4,0,0,0,0,0,0,0,0,1,1,...,1,0,1,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6853,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
6854,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
6855,0,0,0,0,0,0,0,0,1,1,...,1,1,1,0,0,1,1,0,0,0
6856,0,0,0,0,0,0,1,0,1,1,...,1,0,0,0,1,0,0,0,0,0
